In [ ]:
import psycopg2

In [ ]:
%load_ext sql

In [ ]:
DB_ENDPOINT = "localhost"
DEFAULT_DB = 'postgres'
DB = 'csv_etl'
DB_USER = 'postgres'
DB_PASSWORD = '1234'
DB_PORT = '5432'

In [ ]:
try:
    conn = psycopg2.connect(
        host=DB_ENDPOINT,
        port=DB_PORT,
        dbname=DEFAULT_DB,
        user=DB_USER,
        password=DB_PASSWORD)
    print("Connection established")
    conn.set_session(autocommit=True)
    print("Connection is in Auto Commit")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Default Postgres database")
    print(e)

In [ ]:
%sql $conn_string

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS dimRiders
(
    rider_key            INT PRIMARY KEY,
    first_name           VARCHAR(50) NOT NULL,
    last_name            VARCHAR(50) NOT NULL,
    address              VARCHAR(100) NOT NULL,
    birthday             DATE NOT NULL,
    account_start_date   DATE NOT NULL,
    account_end_date     DATE,
    is_member            BOOLEAN
);

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS dimStation
(
    station_key          VARCHAR(50) PRIMARY KEY,
    station_name         VARCHAR(75) NOT NULL,
    latitude             FLOAT NOT NULL,
    longitude            FLOAT NOT NULL
);

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS FactTime
(
    trip_key             VARCHAR(50) PRIMARY KEY,
    rideable_type        VARCHAR(75) NOT NULL,
    start_date_id        INT NOT NULL REFERENCES dimDate(date_key),
    ended_date_id        INT NOT NULL REFERENCES dimDate(date_key),
    start_station_id     VARCHAR(50) NOT NULL REFERENCES dimStation(station_key),
    end_station_id       VARCHAR(50) NOT NULL REFERENCES dimStation(station_key),
    rider_id             INT REFERENCES dimRiders(date_key),
);

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS dimDatePayment
(
    date_key            SERIAL PRIMARY KEY,
    date                DATE,
    year                SMALLINT,
    quarter             SMALLINT,
    month               SMALLINT,
    day                 SMALLINT,
    week                SMALLINT,
    hour                SMALLINT,
    min                 SMALLINT,
    is_weekend          BOOLEAN,
    payment_id          INT
);

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS dimDateTrip
(
    date_key            SERIAL PRIMARY KEY,
    date                DATE,
    year                SMALLINT,
    quarter             SMALLINT,
    month               SMALLINT,
    day                 SMALLINT,
    week                SMALLINT,
    hour                SMALLINT,
    min                 SMALLINT,
    is_weekend          BOOLEAN,
    trip_id             VARCHAR(50)
);

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS FactMoney
(
    payment_id           INT PRIMARY KEY,
    date_id              INT NOT NULL REFERENCES dimDate(date_key),
    rider_id             INT REFERENCES dimRiders(date_key),
    amount               MONEY
);

In [ ]:
%%sql
INSERT INTO dimDatePayment (date_key, date, year, quarter, month, day, week, hour, min, is_weekend)
SELECT payment_id                                           AS payment_id
       DATETIME(date)                                       AS date,
       EXTRACT(year FROM date)                              AS year,
       EXTRACT(quarter FROM date)                           AS quarter,
       EXTRACT(month FROM date)                             AS month,
       EXTRACT(day FROM date)                               AS day,
       EXTRACT(week FROM date)                              AS week,
       EXTRACT(hour FROM date)                              AS hour,
       EXTRACT(minute FROM date)                            AS min,
       CASE WHEN EXTRACT(ISODOW FROM date) IN (6, 7) THEN true ELSE false END AS is_weekend
FROM payment;

In [ ]:
%%sql
INSERT INTO dimDateTrip (date, year, quarter, month, day, week, hour, min, is_weekend)
SELECT trip_id                                                  AS trip_id,
       DATETIME(start_at)                                       AS date,
       EXTRACT(year FROM start_at)                              AS year,
       EXTRACT(quarter FROM start_at)                           AS quarter,
       EXTRACT(month FROM start_at)                             AS month,
       EXTRACT(day FROM start_at)                               AS day,
       EXTRACT(week FROM start_at)                              AS week,
       EXTRACT(hour FROM start_at)                              AS hour,
       EXTRACT(minute FROM start_at)                            AS min,
       CASE WHEN EXTRACT(ISODOW FROM start_at) IN (6, 7) THEN true ELSE false END AS is_weekend
FROM trip;

In [ ]:
%%sql
INSERT INTO dimDateTrip (date_key, date, year, quarter, month, day, week, hour, min, is_weekend)
SELECT trip_id                                                  AS trip_id,
       DATETIME(ended_at)                                       AS date,
       EXTRACT(year FROM ended_at)                              AS year,
       EXTRACT(quarter FROM ended_at)                           AS quarter,
       EXTRACT(month FROM ended_at)                             AS month,
       EXTRACT(day FROM ended_at)                               AS day,
       EXTRACT(week FROM ended_at)                              AS week,
       EXTRACT(hour FROM ended_at)                              AS hour,
       EXTRACT(minute FROM ended_at)                            AS min,
       CASE WHEN EXTRACT(ISODOW FROM ended_at) IN (6, 7) THEN true ELSE false END AS is_weekend
FROM trip;

In [ ]:
%%sql
INSERT INTO dimRiders (rider_key, first_name, last_name, address, birthday, account_start_date, account_end_date, is_member)
SELECT rider_id                                       AS rider_key,
       first                                          AS first_name,
       last                                           AS last_name,
       address                                        AS address,
       birthday                                       AS birthday,
       account_start_date                             AS account_start_date,
       account_end_date                               AS account_end_date,
       is_member                                      AS is_member
FROM riders;

In [ ]:
%%sql
INSERT INTO dimStation (station_key, station_name, latitude, longitude)
SELECT station_id                                     AS station_key,
       name                                           AS station_name,
       latitude                                       AS latitude,
       longitude                                      AS longitude
FROM station;

In [ ]:
%%sql
INSERT INTO FactMoney (payment_id, date_id, rider_id, amount)
SELECT p.payment_id                                         AS payment_id,
       d.date_key                                           AS date_id,
       p.rider_id                                           AS rider_id,
       p.amount                                             AS amount
FROM payment p
JOIN dimDatePayment d ON ( p.payment_id = d.payment_id );

In [ ]:
%%sql
INSERT INTO FactTime (trip_key, rideable_type, start_date_id, ended_date_id, start_station_id, end_station_id, rider_id)
SELECT t.trip_id                                            AS trip_key,
       t.rideable_type                                      AS rideable_type
       ds.date_key                                          AS start_date_id,
       de.date_key                                          AS ended_date_id,
       t.start_station_id                                   AS start_station_id,
       t.end_station_id                                     AS end_station_id,
       t.rider_id                                           AS rider_id
FROM trip t
JOIN dimDateTrip ds ON ( ds.trip_id = t.trip_id ) AND  ( ds.date = DATETIME(t.start_at) )
JOIN dimDateTrip de ON ( de.trip_id = t.trip_id ) AND  ( de.date = DATETIME(t.ended_at) )